In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Bob Seger,You Know Who You Are,It looked like it \r\nIt felt like it \r\nBu...
1,Kenny Rogers,Lay It Down,Traveling down a different road \r\nTrying ha...
2,Linda Ronstadt,1917,The strange young man who comes to me \r\nA s...
3,Glen Campbell,Where Do You Go,Where do you go when there's nowhere to go \r...
4,Dusty Springfield,I Don't Think We Could Ever Be Friends,It's the end of our journey \r\nWhat can I do...
5,Natalie Imbruglia,One More Addiction,First the good news \r\nIt's gonna feel very ...
6,Britney Spears,Just Like Me,[Verse 1] \r\nGot those heels on that you lov...
7,Face To Face,Prodigal,he's never going back refuses to be left behin...
8,Roy Orbison,In Dreams,A candy-colored clown they call the sandman \...
9,John Denver,Durango Mountain Caballero,This song was first released on the Rocky Moun...


In [9]:
df['text'][0]

"It looked like it  \r\nIt felt like it  \r\nBut in the end it was just a show  \r\nWith good acting  \r\nFine acting  \r\nToo bad the sponsers had to go  \r\nAnd you know who you are  \r\nPoor production  \r\nPoor direction  \r\nAll the beautiful sad cliche's  \r\nThe sound was bad  \r\nI couldn't hear you  \r\nBut then what were we doing on the stage  \r\nAnd you know who you are  \r\nOh yeah  \r\nTell me if it feels like the key  \r\nWell it's easy too easy  \r\nCause when I saw you there  \r\nWith your eyes in the air  \r\nAhh  \r\nEverything we done we done before  \r\nSomehow it seemed to make a little more sense  \r\nIt was such a close call I think that's why I'm writing this at all  \r\nI'd call out your name loud  \r\nYou really don't think it would make any sense  \r\nCause you know who you are  \r\n  \r\n  \r\n\r\n"

In [10]:
df.shape

(5000, 3)

In [11]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [18]:
similarity[0]

array([1.        , 0.01068929, 0.03788964, ..., 0.01872954, 0.09577691,
       0.07445179])

In [19]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [20]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [23]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))